In [1]:
import numpy as np
import cv2 as cv
import napari

In [2]:
img = cv.imread('DE_annotated_pictures/20220202_547 D4 001_ch00_yn.tif')
viewer = napari.Viewer()
viewer.add_image(img, rgb=True)

<Image layer 'img' at 0x2ecd8fd3670>

In [3]:
gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
viewer.add_image(gray_img)

<Image layer 'gray_img' at 0x2ecd8fd3370>

In [4]:
low = np.array([0, 0, 0])
high = np.array([130, 130, 120])
mask_inRange = cv.inRange(img, low, high)
masked_img = cv.bitwise_and(img, img, mask=mask_inRange)
viewer.add_image(mask_inRange)

<Image layer 'mask_inRange' at 0x2ecd750ef80>

In [5]:
twoDimage = img.reshape((-1,3))
twoDimage = np.float32(twoDimage)

In [6]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
attempts = 10

ret1, label1, center1 = cv.kmeans(twoDimage, 2, None, criteria, attempts, cv.KMEANS_PP_CENTERS)
ret2, label2, center2 = cv.kmeans(twoDimage, 3, None, criteria, attempts, cv.KMEANS_PP_CENTERS)
ret3, label3, center3 = cv.kmeans(twoDimage, 4, None, criteria, attempts, cv.KMEANS_PP_CENTERS)

center1 = np.uint8(center1)
res1 = center1[label1.flatten()]
mask_k2n = res1.reshape((img.shape))

center2 = np.uint8(center2)
res2 = center2[label2.flatten()]
mask_k3n = res2.reshape((img.shape))

center3 = np.uint8(center3)
res3 = center3[label3.flatten()]
mask_k4n = res3.reshape((img.shape))

viewer.add_image(mask_k2n)
viewer.add_image(mask_k3n)
viewer.add_image(mask_k4n)

<Image layer 'mask_k4n' at 0x2ec822dfa60>

In [7]:
_, thresh = cv.threshold(gray_img, np.mean(gray_img), 255, cv.THRESH_BINARY_INV)
edges = cv.dilate(cv.Canny(thresh, 0, 255), None)

In [8]:
viewer.add_image(thresh)
viewer.add_image(edges)

<Image layer 'edges' at 0x2ec82339540>

In [9]:
kernel1 = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3,3))
kernel2 = cv.getStructuringElement(cv.MORPH_ELLIPSE, (15,15))

opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel1)
closing = cv.morphologyEx(opening, cv.MORPH_CLOSE, kernel2)

In [10]:
viewer.add_image(opening)

<Image layer 'opening' at 0x2ec998c6dd0>

In [11]:
th2 = cv.adaptiveThreshold(gray_img, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 25, 10)
th3 = cv.adaptiveThreshold(gray_img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 51, -10)

In [12]:
viewer.add_image(th2)
viewer.add_image(th3)

<Image layer 'th3' at 0x2ec8c0442e0>

In [13]:
from skimage.filters import threshold_otsu

th_otsu = threshold_otsu(gray_img)
img_otsu  = gray_img < th_otsu

viewer.add_image(img_otsu)

<Image layer 'img_otsu' at 0x2ec99ac8e80>